In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df=pd.read_csv("/content/drive/MyDrive/LGD-model/WOE_calc.csv")

In [4]:
df.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,home_ownership,annual_inc,verification_status,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag,term_int,mths_since_issue_d,mths_since_earliest_cr_line,good_bad
0,0,2500,2500,2500.0,13.56,84.92,C,RENT,55000.0,Not Verified,...,16901.0,36500.0,18124.0,N,Cash,N,36,64.0,200.0,1
1,1,30000,30000,30000.0,18.94,777.23,D,MORTGAGE,90000.0,Source Verified,...,99468.0,15000.0,94072.0,N,Cash,N,60,64.0,366.0,1
2,2,5000,5000,5000.0,17.97,180.69,D,MORTGAGE,59280.0,Source Verified,...,11749.0,13800.0,10000.0,N,Cash,N,36,64.0,80.0,1
3,3,4000,4000,4000.0,18.94,146.51,D,MORTGAGE,92000.0,Source Verified,...,36151.0,5000.0,44984.0,N,Cash,N,36,64.0,142.0,1
4,4,30000,30000,30000.0,16.14,731.78,C,MORTGAGE,57250.0,Not Verified,...,29674.0,9300.0,32332.0,N,Cash,N,60,64.0,204.0,1


In [5]:
df=df.drop("Unnamed: 0",axis=1)

In [6]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991265 entries, 0 to 1991264
Data columns (total 78 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   loan_amnt                    int64  
 1   funded_amnt                  int64  
 2   funded_amnt_inv              float64
 3   int_rate                     float64
 4   installment                  float64
 5   grade                        object 
 6   home_ownership               object 
 7   annual_inc                   float64
 8   verification_status          object 
 9   loan_status                  object 
 10  pymnt_plan                   object 
 11  purpose                      object 
 12  dti                          float64
 13  delinq_2yrs                  float64
 14  inq_last_6mths               float64
 15  open_acc                     float64
 16  pub_rec                      float64
 17  revol_bal                    int64  
 18  revol_util                   float64
 19  

In [13]:
pd.set_option('display.max_columns', 200)

In [14]:
df.isnull().sum()

loan_amnt                      0
funded_amnt                    0
funded_amnt_inv                0
int_rate                       0
installment                    0
                              ..
debt_settlement_flag           0
term_int                       0
mths_since_issue_d             0
mths_since_earliest_cr_line    0
good_bad                       0
Length: 78, dtype: int64

In [15]:
df1=df.copy()

In [18]:
loan_data_defaults = df1[df1['loan_status'].isin(['Charged Off', 'Default','Late (31-120 days)',"In Grace Period"])]


In [19]:
loan_data_defaults.shape

(248779, 78)

In [20]:
pd.options.display.max_rows = None

In [21]:
loan_data_defaults.isnull().sum()

loan_amnt                      0
funded_amnt                    0
funded_amnt_inv                0
int_rate                       0
installment                    0
grade                          0
home_ownership                 0
annual_inc                     0
verification_status            0
loan_status                    0
pymnt_plan                     0
purpose                        0
dti                            0
delinq_2yrs                    0
inq_last_6mths                 0
open_acc                       0
pub_rec                        0
revol_bal                      0
revol_util                     0
total_acc                      0
initial_list_status            0
out_prncp                      0
out_prncp_inv                  0
total_pymnt                    0
total_pymnt_inv                0
total_rec_prncp                0
total_rec_int                  0
total_rec_late_fee             0
recoveries                     0
collection_recovery_fee        0
last_pymnt

In [22]:
loan_data_defaults['recovery_rate'] = loan_data_defaults['recoveries'] / loan_data_defaults['funded_amnt']

In [23]:
loan_data_defaults['recovery_rate'].describe()

count    248779.000000
mean          0.065930
std           0.090608
min           0.000000
25%           0.000000
50%           0.042720
75%           0.104770
max           1.220774
Name: recovery_rate, dtype: float64

In [24]:
# Check how many samples have a recovery rate > 1. This seems to be a data issue
loan_data_defaults[loan_data_defaults['recovery_rate'] > 1].shape

(45, 79)

In [25]:
# Set recovery rates that are greater than 1 to 1 and recovery rates that are less than 0 to 0.
loan_data_defaults['recovery_rate'] = np.where(loan_data_defaults['recovery_rate'] > 1, 1, loan_data_defaults['recovery_rate'])
loan_data_defaults['recovery_rate'] = np.where(loan_data_defaults['recovery_rate'] < 0, 0, loan_data_defaults['recovery_rate'])


In [26]:
# Sanity check
loan_data_defaults['recovery_rate'].describe()

count    248779.000000
mean          0.065921
std           0.090510
min           0.000000
25%           0.000000
50%           0.042720
75%           0.104770
max           1.000000
Name: recovery_rate, dtype: float64

In [27]:
#For EAD model
# Compute dependent variable for the EAD model: credit conversion factor

# credit conversion factor is ratio of the difference of the total funded amount and princial repaid over total funcded amount

loan_data_defaults['CCF'] = (loan_data_defaults['funded_amnt'] - loan_data_defaults['total_rec_prncp']) / loan_data_defaults['funded_amnt']

In [28]:
#sanity check
loan_data_defaults['CCF'].describe()

count    248779.000000
mean          0.690865
std           0.220236
min           0.000000
25%           0.557908
50%           0.741869
75%           0.867180
max           1.000000
Name: CCF, dtype: float64

In [29]:
#Data Prep for LGD EAD Models

In [30]:
categorical=loan_data_defaults.select_dtypes(include=['object']).columns.tolist()

In [31]:
loan_data_defaults=loan_data_defaults.drop(["good_bad","policy_code"],axis=1)

In [32]:
#dummies for cat variables
#Create a dict. object 'd' to store all dummy variable data frames
d = {name: pd.get_dummies(loan_data_defaults[name], prefix=name) for name in categorical}

#deleting last column of each dataframe to remove multi-collinearity
for i in range(len(categorical)-1):
  d[categorical[i]].drop(d[categorical[i]].columns[d[categorical[i]].shape[1]-1], axis=1, inplace=True)

#Concat all dummy data frames
d = pd.concat(d, axis=1)

#Remove categorical variables from the training dataset
loan_data_defaults.drop(categorical, axis=1, inplace=True)

#Concatenate all the dummy variables to the training data set and store into a new data frame
df_model = pd.concat([loan_data_defaults, d], axis=1)

df_model.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_int,mths_since_issue_d,mths_since_earliest_cr_line,recovery_rate,CCF,"(grade, grade_A)","(grade, grade_B)","(grade, grade_C)","(grade, grade_D)","(grade, grade_E)","(grade, grade_F)","(home_ownership, home_ownership_ANY)","(home_ownership, home_ownership_MORTGAGE)","(home_ownership, home_ownership_NONE)","(home_ownership, home_ownership_OTHER)","(home_ownership, home_ownership_OWN)","(verification_status, verification_status_Not Verified)","(verification_status, verification_status_Source Verified)","(loan_status, loan_status_Charged Off)","(loan_status, loan_status_Default)","(loan_status, loan_status_In Grace Period)","(pymnt_plan, pymnt_plan_n)","(purpose, purpose_car)","(purpose, purpose_credit_card)","(purpose, purpose_debt_consolidation)","(purpose, purpose_home_improvement)","(purpose, purpose_house)","(purpose, purpose_major_purchase)","(purpose, purpose_medical)","(purpose, purpose_moving)","(purpose, purpose_other)","(purpose, purpose_renewable_energy)","(purpose, purpose_small_business)","(purpose, purpose_vacation)","(initial_list_status, initial_list_status_f)","(application_type, application_type_Individual)","(hardship_flag, hardship_flag_N)","(disbursement_method, disbursement_method_Cash)","(debt_settlement_flag, debt_settlement_flag_N)","(debt_settlement_flag, debt_settlement_flag_Y)"
3948,11000,11000,11000.0,16.91,272.85,97000.0,28.79,0.0,1.0,15.0,0.0,30642,31.7,32.0,10882.15,10882.15,190.18,190.18,117.85,72.33,0.0,0.0,0.0,210.85,0.0,0.0,0.0,578672.0,96700.0,5.0,38578.0,61687.0,32.5,0.0,0.0,160.0,202.0,9.0,4.0,7.0,9.0,0.0,8.0,10.0,10.0,12.0,10.0,12.0,15.0,10.0,15.0,0.0,0.0,4.0,100.0,30.0,0.0,0.0,650969.0,91197.0,91400.0,65615.0,60,64.0,190.0,0.0,0.989286,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
4330,7000,7000,6975.0,14.47,240.85,35000.0,23.08,0.0,0.0,8.0,0.0,5831,36.4,16.0,6384.09,6361.29,663.75,661.38,615.91,47.84,0.0,0.0,0.0,675.00,0.0,0.0,0.0,12630.0,16000.0,2.0,1579.0,3701.0,57.5,0.0,0.0,42.0,105.0,10.0,10.0,0.0,10.0,0.0,3.0,6.0,3.0,3.0,3.0,7.0,13.0,6.0,8.0,0.0,0.0,1.0,87.5,33.3,0.0,0.0,30399.0,12630.0,8700.0,14399.0,36,64.0,93.0,0.0,0.912013,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0
8335,10000,10000,10000.0,16.14,352.27,82500.0,16.33,4.0,0.0,20.0,0.0,9529,43.1,34.0,9782.23,9782.23,329.85,329.85,217.77,112.08,0.0,0.0,0.0,352.27,0.0,0.0,0.0,318691.0,22100.0,2.0,15935.0,4135.0,65.8,0.0,0.0,212.0,186.0,5.0,5.0,3.0,5.0,2.0,7.0,10.0,7.0,7.0,17.0,12.0,14.0,10.0,20.0,0.0,2.0,1.0,83.9,42.9,0.0,0.0,357233.0,107228.0,12100.0,99651.0,36,64.0,200.0,0.0,0.978223,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,0
8527,34000,34000,34000.0,16.14,1197.70,20000.0,54.86,0.0,0.0,8.0,0.0,33867,59.6,19.0,33259.60,33259.60,1121.48,1121.48,740.40,381.08,0.0,0.0,0.0,1197.70,0.0,0.0,0.0,565988.0,56800.0,3.0,70749.0,10933.0,75.6,0.0,0.0,161.0,200.0,12.0,6.0,1.0,12.0,0.0,4.0,4.0,4.0,5.0,7.0,6.0,11.0,4.0,8.0,0.0,0.0,2.0,100.0,75.0,0.0,0.0,598160.0,65166.0,44800.0,31360.0,36,64.0,188.0,0.0,0.978224,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,

In [33]:
df_model.to_csv("LGD_train_data.csv")